In [1]:
import numpy as np
import pandas as pd
import os
import keras
from keras import models, layers

In [2]:
data_dir = 'F:\study\ml\DataSet\jena_climate'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

In [3]:
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [10]:
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

In [11]:
mean = float_data[:20000].mean(axis=0)
float_data -= mean
std = float_data[:20000].std(axis=0)
float_data /= std

In [100]:
def generator(data,
              lookback,
              delay,
              min_index,
              max_index,
              shuffle=False,
              batch_size=128,
              step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback,
                                     max_index,
                                     size=batch_size)
            print(rows)
            print(len(rows))
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        print(samples.shape)

        targets = np.zeros((len(rows), ))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            print(indices)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
            print(samples.shape)
        yield samples, targets

In [113]:
np.random.randint(2, 7, 3)

array([5, 3, 4])

In [110]:
a = np.random.random((2, 3, 4))
a

array([[[0.66379838, 0.52166437, 0.70167296, 0.53292466],
        [0.67797302, 0.56062222, 0.79461788, 0.93759746],
        [0.92717047, 0.77364219, 0.29450879, 0.32919814]],

       [[0.1807454 , 0.63258905, 0.26108349, 0.09589945],
        [0.21823447, 0.00895664, 0.30849985, 0.90946098],
        [0.97995073, 0.61069926, 0.27274515, 0.23121399]]])

In [95]:
for i, j in enumerate(a):
    print(i)
    print(j)

0
[[0.4157397  0.25283885 0.82567566 0.17590489]
 [0.38415228 0.26206639 0.44080131 0.06687373]
 [0.22731703 0.48989545 0.55832752 0.74522059]]
1
[[0.27013614 0.28234341 0.88327719 0.98167757]
 [0.3096991  0.99353374 0.3418816  0.09295171]
 [0.74899243 0.52581087 0.99673884 0.50757455]]


In [102]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [103]:
train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)

In [104]:
next(train_gen)

[ 99014  30082 133806 186692 182692 133887 109101 199024  35207 134681
  45362  22617  22477  35358  24555 161268   1443  66108  64950 179140
  45038  74529  14681 151532 116428 182998  78146 145045  93256  21932
 140733 172414  34390  52520  26666 176370 111972  27583 178281  17804
   6938 150348  29011  75000 153545  23467  99541  59625  70746  33120
  63591  54094  62478  93243 188351   1442  67110   4112   4132 109688
 134759 122357 157249  89952  65573  45387 183616 122289  53997  71024
  18658  81687 116261 178202 192301 148330 189853 124307 169935 117870
 197894  41081  88551  17052  74829  17887  97225  59501  90184 184333
   4930  86862 152481  87918 163350  46975 109574 115752 105483 167623
  47798  71166  68136 130953 157924  27004  49437 105299 178987 155587
  95262 185286   5320  37138  84733 130680 157848 140140 170634  70487
  40612 167909  16898 118528  20228 119925  60110 169823]
128
(128, 240, 14)
range(97574, 99014, 6)
(128, 240, 14)
range(28642, 30082, 6)
(128, 240,

(array([[[-2.76548754e+00, -1.39990938e-01,  1.62909220e-01, ...,
          -8.88668742e-02, -4.03591699e-01, -1.47112116e+00],
         [-2.81243596e+00, -1.50455883e-01,  1.59076846e-01, ...,
          -6.91408375e-03, -1.89509777e-01, -1.65224750e+00],
         [-2.85746812e+00, -2.04088724e-01,  1.10533447e-01, ...,
           4.98461457e-01,  5.28556669e-01, -1.38492924e+00],
         ...,
         [-4.86093834e-01, -8.37418606e-02, -3.12643761e-02, ...,
          -7.17171601e-01, -8.85276023e-01, -1.14854592e+00],
         [-4.87051965e-01, -1.16444813e-01, -6.32008228e-02, ...,
          -8.05953790e-01, -1.06367762e+00, -1.84435798e+00],
         [-4.96633275e-01, -5.49632628e-02, -6.05387283e-04, ...,
          -1.00400637e+00, -1.04583746e+00,  3.06629346e-01]],
 
        [[-1.45957321e-01,  1.16943526e+00,  1.16060381e+00, ...,
           7.23831631e-01,  4.79496229e-01,  9.14119905e-01],
         [-1.03799556e-01,  1.13934854e+00,  1.12738991e+00, ...,
          -1.10644736

In [99]:
128 * 6

768

In [107]:
(74529 - 73089) / 6

240.0

In [115]:
range(1, 4)

range(1, 4)

In [116]:
np.arange(1, 4)

array([1, 2, 3])

In [120]:
def generator2(data,
               lookback,
               delay,
               min_index,
               max_index,
               shuffle=False,
               batch_size=128,
               step=6):
    i = min_index + lookback
    if max_index is None:
        max_index = len(data) - 1 - delay

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index,
                                     batch_size)
        else:
            if i + batch_size > max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))

        targets = np.zeros((len(rows), ))

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [123]:
train_gen = generator2(float_data,
                       lookback=lookback,
                       delay=delay,
                       min_index=0,
                       max_index=200000,
                       shuffle=True,
                       step=step,
                       batch_size=batch_size)

In [124]:
val_gen = generator2(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    #                       shuffle=True,
    step=step,
    batch_size=batch_size)

In [125]:
test_gen = generator2(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=300001,
    max_index=None,
    #                       shuffle=True,
    step=step,
    batch_size=batch_size)

In [129]:
val_steps = (300000 - 200001 - lookback) // batch_size

In [130]:
test_steps = (len(float_data) - 300001 - lookback) // batch_size

In [131]:
val_steps

769

In [132]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

In [133]:
evaluate_naive_method()

0.33551756286169604


In [134]:
from keras import layers, models, optimizers

In [136]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

In [137]:
model.compile(optimizer=optimizers.RMSprop(), loss='mae')

In [139]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 11s 22ms/step - loss: 1.4357 - val_loss: 0.6586
Epoch 2/20
500/500 [==============================] - 11s 22ms/step - loss: 0.5072 - val_loss: 0.3787
Epoch 3/20
500/500 [==============================] - 11s 22ms/step - loss: 0.3326 - val_loss: 0.4122
Epoch 4/20
500/500 [==============================] - 11s 22ms/step - loss: 0.2980 - val_loss: 0.4143
Epoch 5/20
500/500 [==============================] - 12s 23ms/step - loss: 0.2855 - val_loss: 0.4036
Epoch 6/20
500/500 [==============================] - 11s 23ms/step - loss: 0.2715 - val_loss: 0.3779
Epoch 7/20
500/500 [==============================] - 11s 22ms/step - loss: 0.2616 - val_loss: 0.3890
Epoch 8/20
500/500 [==============================] - 11s 22ms/step - loss: 0.2532 - val_loss: 0.3938
Epoch 9/20
500/500 [==============================] - 11s 22ms/step - loss: 0.2474 - val_loss: 0.3985
Epoch 10/20
500/500 [==============================] - 11s 22ms/step - loss: 0.242

In [141]:
model2 = models.Sequential()
model2.add(layers.GRU(32,
                      input_shape=(lookback // step, float_data.shape[-1])))
model2.add(layers.Dense(1))

In [142]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 32)                4512      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 4,545
Trainable params: 4,545
Non-trainable params: 0
_________________________________________________________________


In [143]:
model2.compile(optimizer=optimizers.RMSprop(), loss='mae')
history2 = model2.fit_generator(train_gen,
                                steps_per_epoch=500,
                                epochs=20,
                                validation_data=val_gen,
                                validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 57s 114ms/step - loss: 0.3560 - val_loss: 0.3208
Epoch 2/20
500/500 [==============================] - 57s 115ms/step - loss: 0.3276 - val_loss: 0.3133
Epoch 3/20
500/500 [==============================] - 56s 113ms/step - loss: 0.3198 - val_loss: 0.3100
Epoch 4/20
500/500 [==============================] - 57s 113ms/step - loss: 0.3162 - val_loss: 0.3060
Epoch 5/20
500/500 [==============================] - 57s 113ms/step - loss: 0.3082 - val_loss: 0.3096
Epoch 6/20
500/500 [==============================] - 58s 115ms/step - loss: 0.3040 - val_loss: 0.3051
Epoch 7/20
500/500 [==============================] - 69s 138ms/step - loss: 0.2984 - val_loss: 0.3079
Epoch 8/20
500/500 [==============================] - 69s 137ms/step - loss: 0.2938 - val_loss: 0.3125
Epoch 9/20
500/500 [==============================] - 63s 127ms/step - loss: 0.2884 - val_loss: 0.3128
Epoch 10/20
500/500 [==============================] - 59s 119ms/step - l

In [145]:
model3 = models.Sequential()
model3.add(
    layers.GRU(32,
               dropout=0.2,
               recurrent_dropout=0.2,
               input_shape=(None, float_data.shape[-1])))
model3.add(layers.Dense(1))
model3.compile(optimizer=optimizers.RMSprop(), loss='mae')

history2 = model3.fit_generator(train_gen,
                                steps_per_epoch=500,
                                epochs=40,
                                validation_data=val_gen,
                                validation_steps=val_steps)

Epoch 1/40
500/500 [==============================] - 84s 168ms/step - loss: 0.4015 - val_loss: 0.3325
Epoch 2/40
500/500 [==============================] - 72s 143ms/step - loss: 0.3660 - val_loss: 0.3199
Epoch 3/40
500/500 [==============================] - 71s 143ms/step - loss: 0.3542 - val_loss: 0.3196
Epoch 4/40
500/500 [==============================] - 67s 133ms/step - loss: 0.3497 - val_loss: 0.3112
Epoch 5/40
500/500 [==============================] - 64s 128ms/step - loss: 0.3445 - val_loss: 0.3182
Epoch 6/40
500/500 [==============================] - 67s 134ms/step - loss: 0.3408 - val_loss: 0.3124
Epoch 7/40
500/500 [==============================] - 64s 129ms/step - loss: 0.3382 - val_loss: 0.3096
Epoch 8/40
500/500 [==============================] - 64s 128ms/step - loss: 0.3366 - val_loss: 0.3074
Epoch 9/40
500/500 [==============================] - 64s 127ms/step - loss: 0.3339 - val_loss: 0.3083
Epoch 10/40
500/500 [==============================] - 64s 127ms/step - l

stacking recurrent layers

In [154]:
model4 = models.Sequential()
model4.add(
    layers.GRU(32,
               dropout=0.1,
               recurrent_dropout=0.5,
               return_sequences=True,
               input_shape=(None, float_data.shape[-1])))
model4.add(
    layers.GRU(64, activation='relu', dropout=0.1, recurrent_dropout=0.5))
model4.add(layers.Dense(1))

In [155]:
model4.compile(optimizer=optimizers.RMSprop(), loss='mae')

history4 = model4.fit_generator(train_gen,
                                steps_per_epoch=500,
                                epochs=40,
                                validation_data=val_gen,
                                validation_steps=val_steps)

Epoch 1/40
500/500 [==============================] - 177s 354ms/step - loss: 0.3907 - val_loss: 0.3392
Epoch 2/40
500/500 [==============================] - 175s 349ms/step - loss: 0.3631 - val_loss: 0.3182
Epoch 3/40
500/500 [==============================] - 174s 348ms/step - loss: 0.3507 - val_loss: 0.3211
Epoch 4/40
500/500 [==============================] - 175s 350ms/step - loss: 0.3439 - val_loss: 0.3247
Epoch 5/40
500/500 [==============================] - 174s 348ms/step - loss: 0.3427 - val_loss: 0.3122
Epoch 6/40
500/500 [==============================] - 174s 348ms/step - loss: 0.3382 - val_loss: 0.3154
Epoch 7/40
500/500 [==============================] - 174s 348ms/step - loss: 0.3352 - val_loss: 0.3116
Epoch 8/40
500/500 [==============================] - 175s 350ms/step - loss: 0.3311 - val_loss: 0.3164
Epoch 9/40
500/500 [==============================] - 174s 348ms/step - loss: 0.3307 - val_loss: 0.3196
Epoch 10/40
500/500 [==============================] - 174s 348m

In [151]:
def reverse_order_generator(data,
                            lookback,
                            delay,
                            min_index,
                            max_index,
                            shuffle=False,
                            batch_size=128,
                            step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback,
                                     max_index,
                                     size=batch_size)
        else:
            if i + batch_size > max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), ))

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples[:, ::-1, :], targets

In [152]:
train_gen_reverse = reverse_order_generator(float_data,
                                            lookback=lookback,
                                            delay=delay,
                                            min_index=0,
                                            max_index=200000,
                                            shuffle=True,
                                            step=step,
                                            batch_size=batch_size)
val_gen_reverse = reverse_order_generator(float_data,
                                          lookback=lookback,
                                          delay=delay,
                                          min_index=200001,
                                          max_index=300000,
                                          step=step,
                                          batch_size=batch_size)

In [153]:
model5 = models.Sequential()
model5.add(layers.GRU(32, input_shape=(None, float_data.shape[-1])))
model5.add(layers.Dense(1))

model5.compile(optimizer=optimizers.RMSprop(), loss='mae')
history = model5.fit_generator(train_gen_reverse,
                               steps_per_epoch=500,
                               epochs=20,
                               validation_data=val_gen_reverse,
                               validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 58s 115ms/step - loss: 0.5523 - val_loss: 0.5694
Epoch 2/20
500/500 [==============================] - 60s 119ms/step - loss: 0.5020 - val_loss: 0.5591
Epoch 3/20
500/500 [==============================] - 58s 115ms/step - loss: 0.4525 - val_loss: 0.5526
Epoch 4/20
500/500 [==============================] - 56s 111ms/step - loss: 0.4103 - val_loss: 0.5634
Epoch 5/20
500/500 [==============================] - 56s 112ms/step - loss: 0.3808 - val_loss: 0.5210
Epoch 6/20
500/500 [==============================] - 60s 119ms/step - loss: 0.3549 - val_loss: 0.5048
Epoch 7/20
500/500 [==============================] - 59s 118ms/step - loss: 0.3353 - val_loss: 0.4767
Epoch 8/20
500/500 [==============================] - 58s 115ms/step - loss: 0.3226 - val_loss: 0.4905
Epoch 9/20
500/500 [==============================] - 57s 114ms/step - loss: 0.3105 - val_loss: 0.4789
Epoch 10/20
500/500 [==============================] - 62s 124ms/step - l

lstm imdb

In [159]:
from keras.datasets import imdb
from keras.preprocessing import sequence

In [157]:
max_features = 10000
maxlen = 500

In [158]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [161]:
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [163]:
model6 = models.Sequential()
model6.add(layers.Embedding(max_features, 128))
model6.add(layers.LSTM(32))
model6.add(layers.Dense(1, activation='sigmoid'))

In [168]:
model6.compile(optimizer=optimizers.RMSprop(),
               loss='binary_crossentropy',
               metrics=['acc'])

history6 = model6.fit(x_train,
                      y_train,
                      epochs=10,
                      batch_size=128,
                      validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 77s 4ms/step - loss: 0.4881 - acc: 0.7683 - val_loss: 0.5242 - val_acc: 0.7714
Epoch 2/10
20000/20000 [==============================] - 79s 4ms/step - loss: 0.3264 - acc: 0.8765 - val_loss: 0.4258 - val_acc: 0.8336
Epoch 3/10
20000/20000 [==============================] - 73s 4ms/step - loss: 0.2685 - acc: 0.9019 - val_loss: 0.3674 - val_acc: 0.8472
Epoch 4/10
20000/20000 [==============================] - 79s 4ms/step - loss: 0.2186 - acc: 0.9211 - val_loss: 0.5078 - val_acc: 0.8468
Epoch 5/10
20000/20000 [==============================] - 89s 4ms/step - loss: 0.1972 - acc: 0.9322 - val_loss: 0.3839 - val_acc: 0.8772
Epoch 6/10
20000/20000 [==============================] - 81s 4ms/step - loss: 0.1736 - acc: 0.9393 - val_loss: 0.3589 - val_acc: 0.8794
Epoch 7/10
20000/20000 [==============================] - 76s 4ms/step - loss: 0.1574 - acc: 0.9456 - val_loss: 0.3604 - val_acc: